# Introdução à MPI - Message Passing Interface
#### _bootcamp_ da Escola Supercomputador Santos Dumont - 2025
por Calebe de Paula Bianchini

### Vamos voltar ao problema do trapézio?

In [ ]:
%%writefile trapezio.c

#include <stdio.h>
#include <math.h>

/* Calcula f(x). */
double f(double x)
{
  double return_val;
  return_val = exp(x);
  return return_val;
}

int main(int argc, char *argv[])
{
  double integral; /* integral armazena resultado final */
  double a, b;     /* a, b -  limite esquerdo e direito da função */
  long i, n;       /* n - número de trapezóides */
  double x, h;     /* h - largura da base do trapezóide */

  a = 0.0;
  b = 1.0;
  n = 800000000; // -> este e um numero bem grande: 8000000000;
  // retire o comentário abaixo para fazer a leitura do valor
  // printf("Entre a, b, e n \n");
  // scanf("%f %f %d", &a, &b, &n);
  h = (b - a) / n;
  integral = (f(a) + f(b)) / 2.0;
  x = a;
  for (i = 1; i < n; i++) {
    x += h;
    integral += f(x);
  }
  integral *= h;
  printf("Com n = %ld trapezóides, a estimativa \n", n);
  printf("da integral de %lf até %lf = %lf\n", a, b, integral);
  return (0);
}

Esse programa pode ser paralelizado facilmente com a decomposição ou
particionamento estático dos dados, assim, cada um dos $p$ processos
ficará responsável por calcular o valor da integral em um subconjunto
consecutivo de $n/p$ trapezoides. Para determinar o valor total da
integral, os valores parciais calculados localmente por cada processo
deverão ser enviados para um único processo, o qual efetuará a soma
desses valores parciais e imprimirá o resultado final. Nesse caso, o
custo de comunicação entre os processos será relativamente baixo, e os
valores *speedup* e eficiência obtidos serão muito altos.

Essa forma de divisão, contudo, necessita que $n$ seja um múltiplo
inteiro de $p$. Se essa exigência não for respeitada, haverá uma
dificuldade para a distribuição de um número igual de trapézios para
todos os processos. O último processo ficará com o resto dessa divisão e
processará um número maior de trapézios.

Uma forma alternativa de particionamento dos dados é determinar que o
primeiro processo calcule a área do primeiro trapézio, o segundo
processo calcule a área do segundo trapézio e assim por diante. Após
calcular a área do primeiro trapézio, o primeiro processo vai calcular a
área de todos os trapézios distantes $p$ trapézios entre si no intervalo
\[a, b\]. Essa nova forma de divisão dos dados tem a vantagem de que $n$
não precisa ser múltiplo de $p$ e a distribuição de carga para cada
processo é mais equânime. Dessa forma, cada processo $q$ calculará
aproximadamente um valor definido pela equação:

$$h * [\displaystyle \sum_{i=1}^{(n-1)/p} f(x_i)]$$

Onde:

$x_1=a + h*(q+1)$, $x_{i+1}=x_i+p*h$, $a < x_i < b$

Esse novo cóidgo pode ser visto a seguir:


In [ ]:
%%writefile mpi_trapezio.c

#include "mpi.h"
#include <stdio.h>
#include <math.h>

double f(double x)
{
  double return_val;
  return_val = exp(x); /* Função exponencial */
  return return_val;
}

int main(int argc, char *argv[])
{
  int meu_ranque, num_procs;         /* respectivamente q e p */
  double a = 0.0, b = 1.0;           /* Limites da integral */
  long int n = 100000000;            /* Número de trapezoides */
  double x, h;                       /* x e h, a base do trapezoide */
  double integral = 0.0, total;      /* Integral de cada processo e total */
  int origem, destino = 0;           /* Origem e destino das mensagens */
  int etiq = 3;                      /* Uma etiqueta qualquer */
  MPI_Init(&argc, &argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
  MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
  /* h é o mesmo para todos os processos */
  h = (b - a) / n;
  /* O processo 0 calcula o valor de f(x)/2 em a e b */
  if (meu_ranque == 0)
  {
    integral = (f(a) + f(b)) / 2.0;
  }

  /* Cada processo calcula a integral sobre n/num_procs trapézios */
  for (x = a + h * (meu_ranque + 1); x < b; x += num_procs * h)
  {
    integral += f(x);
  }
  integral = integral * h;

  /* O processo 0 soma as integrais parciais recebidas */
  if (meu_ranque == 0)
  {
    total = integral;
    for (origem = 1; origem < num_procs; origem++)
    {
      MPI_Recv(&integral, 1, MPI_DOUBLE, origem, etiq, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
      total += integral;
    }
  }
  else
  {
    /* As integrais parciais são enviadas para o processo 0 */
    MPI_Send(&integral, 1, MPI_DOUBLE, destino, etiq, MPI_COMM_WORLD);
  }
  /* Imprime o resultado */
  if (meu_ranque == 0)
  {
    printf("Com n = %ld trapezoides, a estimativa\n", n);
    printf("da integral de %lf até %lf = %lf \n", a, b, total);
  }
  MPI_Finalize();
  return (0);
}

Sua compilação e execução ficariam:
- não se esqueçam de configurar o ambiente!

In [ ]:
!mpicc mpi_trapezio.c -o mpi_trapezio -Wall -O3 -lm
!time -p mpirun --allow-run-as-root --host localhost:4 -np 4 ./mpi_trapezio

Maiores detalhes sobre Programação Paralela e MPI, vejam:

* __Programação Paralela e Distribuída__ _com MPI, OpenMP e OpenACC para computação de alto desempenho_, em [Casa do Código](https://www.casadocodigo.com.br/products/livro-programacao-paralela).

